# Project Discrete Optimization and Decision Making

## First point

This project involves designing an efficient last-mile delivery system for a city. A company has a set of customers $C = \{1, \ldots, \overline{c}\}$, each requiring the delivery of a package. The weight of each package $w_c, c \in C$ is known. The delivery time for each customer is equal to $s_c, c \in C$. The company has a set $K = \{1, \ldots, \overline{k}\}$ of delivery vans available that all start from the same depot $0$ at time $0$, have the same capacity $W$, and must return to the depot withing $t_{max}$. Each vehicle can exit the depot at most once. The problem can be formulated on a complete directed graph $G = (V, A)$ where $V = C \cup \{0\}$ is the set of nodes and $A = \{(i,j) | i, j \in V, i \neq j\}$ is the set of arcs. Each arc can be traveled at most once by any of the vehicles. For each arc $(i, j) \in A$, let us define the time $t_{ij}$ required to travel over the arc. Travel times satisfy the triangle inequality. The goal of the company is to minimize the total time required to complete the service for all customers. Provide a mathematical formulation of the problem and its optimal solution. 

### Mathematical Model

In [1]:
import gurobipy as gp
from gurobipy import GRB

#### Constants

* $C = \{1, \ldots, \overline{c}\}$: Set of customers.
* $K = \{1, \ldots, \overline{k}\}$: Set of delivery vans.
* $V = C \cup \{0\}$: Set of nodes (customers and depot).
* $A = \{(i,j) | i,j \in V, i \neq j\}$: Set of arcs.
* $w_c$: Weight of the package for customer $c$.
* $s_c$: Delivery time for customer $c$.
* $t_{ij}$: Travel time from node $i$ to node $j$.
* $W$: Capacity of each vehicle.
* $t_{max}$: Maximum allowed time for a vehicle to be out of the depot

In [2]:
# Sets and parameters
C = range(1, 3)  # Example set of customers
K = range(1, 3)  # Example set of vans
V = [0] + list(C)  # Set of nodes including the depot (0)
W = 20  # Capacity of each van
t_max = 100  # Maximum allowed time for each van

w_c = {c: 10 for c in C}  # Weights of packages
s_c = [0,10,10]  # Delivery times
t_ij = {(i, j): 1 for i in V for j in V if i != j}  # Travel times

In [3]:
# Create a new model
model = gp.Model("last_mile_delivery")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-10


#### Variables

* $x_{ij}^k$: A binary variable that is 1 if vehicle k travels from node i to node j, and 0 otherwise.
* $t_i$: The time at which service starts at node i.
* $y_k$: A binary variable that is 1 if vehicle k is used, and 0 otherwise.

In [4]:
x = model.addVars(V, V, K, vtype=GRB.BINARY, name="x")
t = model.addVars(V, vtype=GRB.CONTINUOUS, name="t")
y = model.addVars(K, vtype=GRB.BINARY, name="y")

#### Model

Minimize the total required to complete the service for all customers.

$ \begin{align}\min\sum_{k\in K}\sum_{i \in V}\sum_{j \in V}t_{ij}x^k_{ij}\end{align}$

In [5]:
model.setObjective(gp.quicksum(t_ij[i, j] * x[i, j, k] for i in V for j in V if i != j for k in K), GRB.MINIMIZE)

Each customer is visited exactly once by one vehicle

$ \begin{align}\sum_{k \in K}\sum_{i \in V}x^k_{ij} = 1 \ \ \ \ \forall j \in C\end{align}$

In [6]:
#Each customer is visited exactly once by one vehicle
model.addConstrs((gp.quicksum(x[i, j, k] for i in V if i != j for k in K) == 1 for j in C), name="visit_once")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

One vehicle enter and One vehicle exit from a node

$ \begin{align}\sum_{j \in V} x^k_{ij} = \sum_{j \in V}x_{ji}^k \ \ \ \ \forall k \in K, \ \forall i \in V \end{align}$

In [7]:
# Flow conservation for each vehicle
model.addConstrs((gp.quicksum(x[i, j, k] for j in V if i != j) == gp.quicksum(x[j, i, k] for j in V if i != j) for k in K for i in V), name="flow_conservation")

{(1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>}

Capacity contraint for each vehicle: a vehicle can't transport more than W kg

$ \begin{align}\sum_{c\in C}w_c \sum_{j\in V}x^k_{cj} \leq W \ \ \ \ \forall k \in K \end{align}$

In [8]:
# Capacity constraint for each vehicle
model.addConstrs((gp.quicksum(w_c[c] * gp.quicksum(x[c, j, k] for j in V if c != j) for c in C) <= W for k in K), name="capacity")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

Maximum route duration for each vehicle:

$ \begin{align}\sum_{i \in V}\sum_{j\in V}t_{ij}x_{ij}^k \leq t_{max} \ \ \ \ \forall k \in K \end{align}$

In [9]:
# Maximum route duration for each vehicle
model.addConstrs((gp.quicksum(t_ij[i, j] * x[i, j, k] for i in V for j in V if i != j) <= t_max for k in K), name="max_duration")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

Service time constraints:

$\begin{align}t_i + s_i + t_{ij} \leq t_j + M(1-x^k_{ij}) \ \ \ \ \forall k \in K, \ \forall (i, j) \in A , \ i \neq j \end{align}$

where M is a large constant

In [10]:
# Service time constraints
M = 1e6  # Large constant
model.addConstrs(( t[i] + s_c[i] + t_ij[i, j]  <= t[j] + M * (1 - x[i, j, k]) for k in K for i in V for j in V if i != j), name="service_time")

{(1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 1): <gurobi.Constr *Awaiting Model Update*>}

Depot constraints

$\begin{align}\sum_{j \in V}x^k_{0j} &= y_k \ \ \ \ \forall k \in K \\
    \sum_{i \in V}x^k_{i0} &= y_k \ \ \ \ \forall k \in K
\end{align} $

In [11]:
# Depot constraints
model.addConstrs((gp.quicksum(x[0, j, k] for j in C) == y[k] for k in K), name="depot_start")
model.addConstrs((gp.quicksum(x[j, 0, k] for j in C) == y[k] for k in K), name="depot_end")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

#### Solution

In [12]:
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    solution = model.getAttr('x', x)
    for k in K:
        print(f"Vehicle {k}:")
        route = [0]
        while route[-1] != 0 or len(route) == 1:
            i = route[-1]
            used = False
            for j in V:
                if i != j and solution[i, j, k] > 0.5:
                    used = True
                    route.append(j)
                    break
            if not used:
                used = False
                break
        print(" -> ".join(map(str, route)))
else:
    print("No optimal solution found")
print(t)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 23 columns and 100 nonzeros
Model fingerprint: 0x244831c7
Variable types: 3 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 2 rows and 6 columns
Presolve time: 0.00s
Presolved: 26 rows, 17 columns, 92 nonzeros
Variable types: 3 continuous, 14 integer (14 binary)

Root relaxation: objective 2.000000e+00, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.00000    0    4          -    2.00000      -     -    0s
     0     0    3.00000    0    8          -  